In [ ]:
!pip install ultralytics
!pip install matplotlib


!wget https://data.mendeley.com/datasets/8h3s6jkyff/1
!unzip 8h3s6jkyff.zip



import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Path to the dataset
data_path = 'https://drive.google.com/drive/folders/1jKLE8hvBiJ3sidIgV5voPnL6vvnCtf7V?usp=drive_link'

# Function to load images and annotations
def load_data(data_path):
    images = []
    annotations = []
    for img_file in os.listdir(data_path):
        if img_file.endswith('.jpg'):
            img = cv2.imread(os.path.join(data_path, img_file))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            images.append(img)
            # Load corresponding annotation file
            annotation_file = img_file.replace('.jpg', '.txt')
            with open(os.path.join(data_path, annotation_file), 'r') as file:
                annotations.append(file.readlines())
    return images, annotations

images, annotations = load_data(data_path)





from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')  # You can also use 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', etc.





# Split the dataset into training and validation sets
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size=0.2, random_state=42)

# Save the train and validation data in YOLO format
def save_yolo_format(images, annotations, save_path):
    for idx, (img, ann) in enumerate(zip(images, annotations)):
        img_path = os.path.join(save_path, f'image_{idx}.jpg')
        ann_path = os.path.join(save_path, f'image_{idx}.txt')
        cv2.imwrite(img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        with open(ann_path, 'w') as file:
            file.writelines(ann)

os.makedirs('data/train', exist_ok=True)
os.makedirs('data/val', exist_ok=True)
save_yolo_format(train_images, train_annotations, 'data/train')
save_yolo_format(val_images, val_annotations, 'data/val')





# Train the model
model.train(data='data.yaml', epochs=50, imgsz=640)






# Evaluate the model on the validation set
results = model.val(data='data.yaml')





# Print results
print("Precision: ", results.metrics['precision'])
print("Recall: ", results.metrics['recall'])
print("mAP@0.5: ", results.metrics['map50'])
print("mAP@0.5:0.95: ", results.metrics['map'])






model.show_results(results, save=True)
